## Compare results for multiple results

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [28]:
%matplotlib widget

In [29]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [30]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [31]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample,img_size):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=img_size
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict
    

In [32]:
### Read validation data
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

bkgnd_dict={}
num_bkgnd=200
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=[0,1,2,3]
### Extract validation data
for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:,:]
    
    dict_val=f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
# del samples


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_1.1_train_val.npy


In [33]:
df_runs=pd.DataFrame([])
img_size=64

In [34]:
### bkgnd: Get bkgnd chi-squares for different sigma wrt one standard sigma 
lst=[0.5,0.65,0.8,1.1]
param_label=0.65
for count,run in enumerate(lst):
    num=200
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(run)
    samples=np.load(fname,mmap_mode='r')[1000:1000+num][:,0,:,:]
    dict1={'label':'ref_%s'%(run),'fname':''}
    dict1['num_imgs']=num

    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(samples,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    
    df_runs=df_runs.append(dict1,ignore_index=True)


### Add pytorch runs

In [36]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'
# fldr='20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50/'
fldr='20210619_224213_cgan_bs32_nodes8_lr0.0001-vary_fm50/'
parent_dir+=fldr
param_label=0.65

for param_label in sigma_lst:
    prefix='pred_%s_'%(param_label)
    lst=[(64,37810),(56,33080),(58,34460)]

    for count,run in enumerate(lst):
        epoch,step=run[0],run[1]
        key=prefix+'{0}-{1}'.format(epoch,step)
        print(key)
        fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
        dict1={'label':key,'fname':fname}
        images=np.load(dict1['fname'])     ### No transform needed here

        images=images[:,0,:,:,:]
        print(images.shape)
        print(np.max(images),np.min(images))

        dict1['num_imgs']=images.shape[0]
        ### Compute spectrum and histograms
        dict_sample=f_compute_hist_spect(images,bins)
        ### Compute chi squares
        dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)

        dict1.update(dict_sample)
        dict1.update(dict_chisqrs)
        del(images)
        df_runs=df_runs.append(dict1,ignore_index=True)


pred_0.5_64-37810
(128, 64, 64, 64)
0.9959554 -0.9936986
pred_0.5_56-33080
(128, 64, 64, 64)
0.991765 -0.9940155
pred_0.5_58-34460
(128, 64, 64, 64)
0.9964063 -0.99557513
pred_0.65_64-37810
(128, 64, 64, 64)
0.99384123 -0.99806195
pred_0.65_56-33080
(128, 64, 64, 64)
0.99575 -0.99600095
pred_0.65_58-34460
(128, 64, 64, 64)
0.99758416 -0.996685
pred_0.8_64-37810
(128, 64, 64, 64)
0.9982989 -0.99808645
pred_0.8_56-33080
(128, 64, 64, 64)
0.99887234 -0.9981009
pred_0.8_58-34460
(128, 64, 64, 64)
0.9971306 -0.99755985
pred_1.1_64-37810
(128, 64, 64, 64)
0.99858075 -0.99908257
pred_1.1_56-33080
(128, 64, 64, 64)
0.9984335 -0.99926037
pred_1.1_58-34460
(128, 64, 64, 64)
0.9987511 -0.99922436


In [37]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_imgs', 'spec_sdev', 'spec_val'],
      dtype='object')

In [38]:
col_list=['label', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [39]:
df_runs

,label,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,ref_0.5,0.086267,0.057220,0.669133,47.354748,0.027979,0.001067,0.026892,0.795546,57.897684,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.004302604552971646, 0.002018451278322593, 0...","[0.853119231874845, 0.5878142331624938, 0.0389...","[1840428.129624967, 659888.2576158947, 319059....","[6553408.145947721, 3140689.315330465, 1684023...",200.0,
1,ref_0.65,0.001755,0.000824,0.020739,39.992178,0.000692,0.000239,0.000171,1.077529,1.399420,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.004845659303598767, 0.002227391406005946, 0...","[1.005117045253277, 0.5011638175556109, 0.0439...","[2423873.7494351915, 866986.1767551795, 397620...","[8112540.717692038, 3777925.4307417036, 198250...",200.0,
2,ref_0.8,0.076040,0.032979,0.228422,47.854804,0.032064,0.010997,0.020829,1.254100,66.992929,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.004629317152962012, 0.0020779417256222535, ...","[1.119366114046578, 0.4366659185693627, 0.0456...","[3109237.00987531, 1106093.839588495, 532441.1...","[9658460.587204015, 4456946.35372482, 2303768....",200.0,
3,ref_1.1,0.512256,0.165835,1.663675,51.099874,0.206514,0.139908,0.110730,1.732093,284.805873,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.005673388457959078, 0.002555358178987679, 0...","[1.2842107376798189, 0.346968812853727, 0.0426...","[4052787.5019684853, 1651760.6682976387, 76359...","[11350881.200524844, 5062233.4895504145, 25918...",200.0,
4,pred_0.5_64-37810,0.008175,0.003203,8.052332,39.638242,0.002540,0.002433,0.000093,1.355345,1547.280109,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.005628915568210718, 0.002692881942000374, 0...","[0.8391317746825848, 0.5949379204725701, 0.042...","[1759710.9986237232, 572308.7334764905, 311778...","[7230280.37264872, 3166212.6309647495, 1696139...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,pred_0.5_56-33080,0.015525,0.009190,8.275993,39.226641,0.005797,0.000538,0.000124,1.323221,1336.722800,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.005793322337990075, 0.0027843682012599425, ...","[0.8417412120208732, 0.5956265122588328, 0.038...","[1726135.8306904517, 582850.5768002907, 293472...","[6250315.273353475, 2900239.8633354907, 158592...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,pred_0.5_58-34460,0.006913,0.003089,8.416034,40.723402,0.001963,0.001861,0.000909,1.380965,1751.650775,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0056185567509620385, 0.002704026728878361, ...","[0.8197771097195806, 0.6049525725689788, 0.043...","[1960045.9598884308, 568551.2772049728, 309830...","[6764901.482090776, 3218823.168060783, 1726574...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,pred_0.65_64-37810,0.029099,0.012526,8.172498,42.426048,0.013927,0.002647,0.001262,1.107261,1289.430460,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0053472620099633454, 0.0025048793650201497,...","[0.9658175107789236, 0.5231099203557, 0.045944...","[2160653.38969704, 647449.0848416795, 357889.2...","[8000913.530029979, 3714317.0797666237, 196614...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,pred_0.65_56-33080,0.055379,0.028347,8.788615,47.026668,0.025961,0.001070,0.001885,1.088319,1444.992451,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.005729683441750843, 0.0027351910325892174, ...","[0.9630340483665393, 0.5276479883482676, 0.040...","[1776351.182466449, 670768.6896995255, 310015....","[6984617.21570887, 3254350.7924500164, 1781460...",128.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,pred_0.65_58-34460,0.043433,0.013172,8.553379,45.056899,0.011800,0.018461,0.002926,1.217775,1520.402366,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.005893426344248796, 0.002789295385977846, 0...","[0.9481703068013765, 0.5322872628367954, 0.046...","[1930890.4333845724, 619538.6852397599,

### Compare chi-squares

In [40]:
chi_keys=['label','chi_1','chi_spec1', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,ref_0.5,0.086267,0.669133,200.0,0.057220,0.027979,0.001067,0.026892,0.795546,57.897684,47.354748
1,ref_0.65,0.001755,0.020739,200.0,0.000824,0.000692,0.000239,0.000171,1.077529,1.399420,39.992178
2,ref_0.8,0.076040,0.228422,200.0,0.032979,0.032064,0.010997,0.020829,1.254100,66.992929,47.854804
3,ref_1.1,0.512256,1.663675,200.0,0.165835,0.206514,0.139908,0.110730,1.732093,284.805873,51.099874
4,pred_0.5_64-37810,0.008175,8.052332,128.0,0.003203,0.002540,0.002433,0.000093,1.355345,1547.280109,39.638242
5,pred_0.5_56-33080,0.015525,8.275993,128.0,0.009190,0.005797,0.000538,0.000124,1.323221,1336.722800,39.226641
6,pred_0.5_58-34460,0.006913,8.416034,128.0,0.003089,0.001963,0.001861,0.000909,1.380965,1751.650775,40.723402
7,pred_0.65_64-37810,0.029099,8.172498,128.0,0.012526,0.013927,0.002647,0.001262,1.107261,1289.430460,42.426048
8,pred_0.65_56-33080,0.055379,8.788615,128.0,0.028347,0.025961,0.001070,0.001885,1.088319,1444.992451,47.026668
9,pred_0.65_58-34460,0.043433,8.553379,128.0,0.013172,0.011800,0.018461,0.002926,1.217775,1520.402366,45.056899


## Plot

In [41]:
def f_plot_hist_spec(df,dict_bkg,plot_type,img_size):

    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
#             yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            yerr2=row.spec_sdev

            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)
    
            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.legend()
            plt.title("relative spectrum")
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:,0]
            f_plot_grid(images[:8],cols=4,fig_size=(8,4))

            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
#         y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']

        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        plt.xlim(0,img_size/2)


    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.title("relative spectrum")
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)    
    
    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.5, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})

    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [47]:
# Define the reference 
param_label=0.8
dict_val=bkgnd_dict[str(param_label)]

In [48]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type,img_size)

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                img_size=fixed(img_size),
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('ref_0.5', 'ref_0.65', 'ref_0.8', 're…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [45]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_imgs,chi_2,chi_spec2
0,ref_0.5,0.086267,0.669133,200.0,0.026892,57.897684
1,ref_0.65,0.001755,0.020739,200.0,0.000171,1.399420
2,ref_0.8,0.076040,0.228422,200.0,0.020829,66.992929
3,ref_1.1,0.512256,1.663675,200.0,0.110730,284.805873
4,pred_0.5_64-37810,0.008175,8.052332,128.0,0.000093,1547.280109
5,pred_0.5_56-33080,0.015525,8.275993,128.0,0.000124,1336.722800
6,pred_0.5_58-34460,0.006913,8.416034,128.0,0.000909,1751.650775
7,pred_0.65_64-37810,0.029099,8.172498,128.0,0.001262,1289.430460
8,pred_0.65_56-33080,0.055379,8.788615,128.0,0.001885,1444.992451
9,pred_0.65_58-34460,0.043433,8.553379,128.0,0.002926,1520.402366


### Test 

In [ ]:
df_runs.fname.values

In [ ]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210112_191125_cgan_predict_1.1_m2/images/inference_spec_label-1.1_epoch-11_step-25999.npy'
# a1=np.load(fname)
# print(a1.shape)

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy'
val_arr=np.load(fname)[:2000][:,0,:,:]
print(val_arr.shape)

In [ ]:
f_pixel_intensity(f_invtransform(a1),normalize=True)
plt.xscale('symlog',linthreshx=50)


# img_lst=[a1]
# f_compare_pixel_intensity(img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


In [ ]:
val_arr=val_arr[:,0,:,:]

In [ ]:
img_lst=[a1,val_arr]
f_compare_spectrum(img_lst=img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


### Test histogram

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy'
print(fname)
samples=np.load(fname,mmap_mode='r')[-1000:][:,0,:,:]
print(samples.shape)